# 4-7. 프로젝트 : 멋진 작사가 만들기

## Step 1. 데이터 다운로드

이미 실습(1) 데이터 다듬기에서 Cloud shell에

심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면,

~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

## Step 2. 데이터 읽어오기

In [1]:
# 필요한 모듈 import 하기 

import glob
import os
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
# print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088


In [3]:
# 공백인 문장은 길이를 검사하여 길이가 0이라면 제외를 시켜 봅니다.

for idx, sentence in enumerate(raw_corpus): # enumerate 열거하다 라는 뜻으로, 리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'


## Step 3. 데이터 정제

## 문장 전처리

### 1. 입력된 문장을 정규 표현식을 이용해서 정제합니다.

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 정규표현식으로 문장이 어떻게 필터링되는지 확인해 봅니다.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 2. preprocess_sentence로 정제한 문장을 모음

In [21]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue  # 길이가 0인 빈 문장은 건너뜁니다
    if sentence[-1] == "]": continue # ']'로 끝나는 파트를 나눈 문장은 건너뜁니다
    if sentence[-1] == ")": continue # ')'로 끝나는 코러스 문장은 건너뜁니다    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence.count(" ") >= 15:
            continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> look like you hidin your problems <end>']

### 3. tokenize() 함수로 데이터를 Tensor로 변환 (Tokenizer와 pad_sequences를 사용)

In [22]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # maxlen=15로 설정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [23]:
tensor, tokenizer = tokenize(corpus) # 토큰화 적용

tensor.shape

[[   2    4   92 ...   10   12    3]
 [   2   37  134 ...    0    0    0]
 [   2    4   36 ...    0    0    0]
 ...
 [   2   87  694 ...    0    0    0]
 [   2  204    3 ...    0    0    0]
 [   2    9 1525 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7febb8280e90>


(148425, 15)

In [24]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력

print(tensor[:3, :10])

[[   2    4   92  104   58   31  164    5   11  134]
 [   2   37  134   30   10   12    3    0    0    0]
 [   2    4   36  798  170 2348  798   38   10   12]]


In [25]:
# tokenizer에 구축된 단어 사전 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break # 단어 사전의 10번째 단어까지 출력
        
print(tokenizer.num_words)

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
12000


#### Source & Target

모델을 학습할 때 입력값(source)과 출력값(target)을 데이터셋 형식으로 정의합니다

- 입력값(source) : Start를 시작으로 맨 뒤값인 End를 제외한 문장(== sentence[:-1])
- 출력값(target) : Start 다음 단어를 시작으로 End까지의 문장(== sentecne[1:])

In [26]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다

# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높으므로, 잘라서 소스 문장을 생성합니다
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2   4  92 104  58  31 164   5  11 134  23  30  10  12]
[  4  92 104  58  31 164   5  11 134  23  30  10  12   3]


## Step 4. 평가 데이터셋 분리

### 1. sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리

In [27]:
# 사이킷 런 패키지를 이용해서 위에서 train과 target을 train과 validation 셋으로 분리를 해줍니다. 8:2로 분리를 하겠습니다.

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

In [28]:
# train의 맨 뒷 단어를 뺀 문장을 분석해서 앞 단어가 없는 target을 찾는것이 목적이기 때문에 shape는 동일한 모습을 하고있습니다.

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (118740, 14)
Target Train: (118740, 14)


### 2. 데이터셋 정의

In [29]:
BUFFER_SIZE = len(src_input) # 텐서의 1차원, 전체 문장의 개수
BATCH_SIZE = 256 # 문장의 개수
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

### 1. 모델 구조 정의

In [30]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

### 2. 모델 생성

## - 하이퍼 파라미터 지정과 모델 생성
    - embedding_size를 512으로 올려 고려하는 feature의 수를 늘림
    - hidden_size를 2048으로 올려 각 레이어의 노드 수를 늘림

In [31]:
embedding_size = 512 # 단어 하나의 특징 수
hidden_size = 2048 # 퍼셉트론의 갯수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [32]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

In [33]:
# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.46774313e-04, -4.96039866e-04,  2.59472090e-05, ...,
          1.27862237e-04,  9.56478834e-05,  2.65176524e-04],
        [-1.50246036e-04, -3.84326471e-04, -5.37159176e-05, ...,
          1.42909921e-04,  1.60650146e-04,  3.43132153e-04],
        [-2.00931652e-04,  2.80696240e-05, -9.22709951e-05, ...,
          2.37725006e-04,  2.54640589e-04,  2.63643131e-04],
        ...,
        [-2.09589000e-03,  1.08527229e-03,  3.34497483e-04, ...,
         -1.03862677e-03, -5.92302822e-04, -3.37539939e-04],
        [-1.95793086e-03,  1.35472941e-03,  4.05756466e-04, ...,
         -1.50318746e-03, -8.96097510e-04, -6.74814393e-04],
        [-1.78136653e-03,  1.58377504e-03,  4.36176226e-04, ...,
         -1.92987768e-03, -1.19189301e-03, -9.71210888e-04]],

       [[ 1.46774313e-04, -4.96039866e-04,  2.59472090e-05, ...,
          1.27862237e-04,  9.56478834e-05,  2.65176524e-04],
        [-1.87158133e-04, -5.12017112e-04,  2

- 생성된 모델 살펴보기: model_summary
    - 하나의 배치만 이용하면 임시 모델을 생성해볼 수 있다

In [34]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  20979712  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_1 (Dense)              multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [35]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)

### 3. 모델 학습: fit

In [36]:
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=256)

Epoch 1/10
464/464 [==============================] - 461s 986ms/step - loss: 3.8308 - val_loss: 2.9309
Epoch 2/10
464/464 [==============================] - 458s 988ms/step - loss: 2.8112 - val_loss: 2.6885
Epoch 3/10
464/464 [==============================] - 458s 988ms/step - loss: 2.4669 - val_loss: 2.5118
Epoch 4/10
464/464 [==============================] - 459s 990ms/step - loss: 2.1527 - val_loss: 2.3791
Epoch 5/10
464/464 [==============================] - 457s 985ms/step - loss: 1.8497 - val_loss: 2.2797
Epoch 6/10
464/464 [==============================] - 457s 985ms/step - loss: 1.5783 - val_loss: 2.2077
Epoch 7/10
464/464 [==============================] - 458s 987ms/step - loss: 1.3557 - val_loss: 2.1738
Epoch 8/10
464/464 [==============================] - 457s 984ms/step - loss: 1.1895 - val_loss: 2.1675
Epoch 9/10
464/464 [==============================] - 457s 985ms/step - loss: 1.0747 - val_loss: 2.1798
Epoch 10/10
464/464 [==============================] - 458s 987m

### 4. 실습 : 잘 만들어졌는지 평가하기

In [37]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

# Step 6. 평가 및 회고

## 평가

### 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?

- 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?

- 모델 학습 후 실습에서 문장을 생성해본 결과 적절한 가사 텍스트를 생성하였음을 확인할 수 있습니다.
   - i love -> i love you , i m not gonna crack

### 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?

- 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?

- 데이터의 전처리: 입력된 문장을 정규 표현식을 이용해서 정제함으로써 특수 문자를 제거하였습니다.

- 데이터셋 구성 과정: 우선, 모델 학습을 위하여 입력값(source)과 출력값(target)을 데이터셋 형식으로 정의한 후, 텐서플러우의 sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 8:2 분리하고, BUFFER_SIZE를 조정 함으로써 데이터셋을 구성하였습니다.
.
- 토크나이저 생성의 경우엔, 노드에 나온 설명대로 토큰 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하는 것을 권고하여서 maxlen=15로 설정하였으며, 패딩 처리를 위해서 padding='post'로 설정하였습니다.

### 3. 텍스트 생성모델이 안정적으로 학습되었는가?

- 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

- batch size, embedding size, hidden size 등의 변수들을 조정해보면서 validation loss를 2.2 이하로 맞추기 위해서 다양한 시도를 진행하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계해 보았습니다.

- 초기에는 변수들을 다양하게 조정을 하며 진행을 해보았으나, 제출 직전에는 다른 변수들은 고정해놓고 embedding size, hidden size 조정을 통해 loss 값을 조정해보았습니다.

- 하지만, Total params가 늘어날수록 모델의 학습 시간이 오래 걸렸을 뿐만 아니라 loss 값이 유의미한 결과에 도달하지 못함을 확인 할 수 있었습니다.

- 결과적으로, batch_size=256, embedding_size=512, hidden_size를 2048로 수정을 한 이후에서야, validation loss가 약 2.2 이하의 결과를 보이는 것을 확인 할 수 있었습니다.

## 회고

### 1. 어려웠던 점

- 노드를 따라서, 학습을 진행할 때, 처음 코드를 보고 이해하는 과정에서 어려움을 느꼈습니다.
- 하지만, 여러 번 반복해서 읽어보면서 각 과정이 의미하는 바를 알아보는 좋은 시간이 되었다고 생각합니다.

- 이번 노드에서 배운 대로 진행을 하였으나, 원하는 loss 값이 나오지 않아서 여러 시행착오를 겪어야 했던 것과 모델 학습 과정에서 많은 시간이 소요되는 과정에서 저의 인내심 부족으로 인해 어려움을 느꼈습니다.

### 2. 배운 점

- 하이퍼파라미터 설정 과정에서 다양한 시도를 해보고 최적의 결과를 내기 위해서 어떻게 해야 하는 지에 대해서 고민할 수 있었던 EXPLORATION이었다고 생각합니다.

### 3. 마무리

- 생각보다 모델의 학습시간을 위한 시간이 많이 소요된다는 생각이 듭니다. 좀 더 효율적으로 모델을 학습시키면서 높은 정확도를 내보고 싶다고 생각하였으나, 어떤 식으로 진행을 해야 좋은 방법인지에 대해서는 조금 더 AIFFEL 에서 학습을 하면서 이를 위해 필요한 것이 무엇이 있을 지 알아보는 시간이 필요할 것으로 보입니다.